# Introduction to the $\beta$-PDF
At a location in a given diffusion flame (i.e. known initial composition, strain, and time scale), the value of any flame parameter $\phi$ (e.g. $T$, $\rho$, etc.) is a function of the mixture fraction, $\xi$. $\xi$ is defined as what fraction of the mass in the flame originated from the fuel, with $\xi = 1$ corresponding to pure fuel. In order to create a tabular version of table parameters, we need to determine the average values of flame parameters. The general formula for the statistical average of a value $\phi = \phi(\xi)$ is given by
$$\int_{-\infty}^{\infty} \phi(\xi)*P(\xi)d\xi$$
where $P(\xi)$ is the probability distribution function desribing the probability of a given value of $\xi$ occurring. In a diffusion flame like the one used to model the flame's chemistry, the distribution of $\xi$ closely matches the shape of the $\beta$-PDF, given by:
$$f(x; \alpha, \beta) = \frac{\Gamma(\alpha + \beta)}{\Gamma(\alpha)\Gamma(\beta)}~x^{\alpha-1}(1-x)^{\beta-1}$$
Qualitatively, the $\beta$-PDF's behavior mathces diffusive mixing:

#### Simulation Code

In [30]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
from scipy.interpolate import interp1d
import LiuInt as LI
from IPython.display import HTML

import importlib
importlib.reload(LI)
%matplotlib inline

if input("Rerun simulation")=='y':
    #----- Model diffusive mixing
    # Parameters
    t_steps = 1000
    n_points = 250
    xmax = 100
    ymax = 10
    partitionLoc = int(xmax/2)  # Location of the partition as a fraction of the total domain
    tLiftPartition = 20         # Time to lift the partition
    maxStep = 3                 # Max step size for particles
    fps = 24                    # Frames per second
    histBins = 10               # Number of bins for the histogram

    # Storage arrays
    points = np.zeros([t_steps, n_points, 2])
    partition = np.ones(t_steps)
    partition[tLiftPartition:] = 0
    xBounds = [0, partitionLoc]
    yBounds = [0,ymax]

    # Initialize the particles in the box
    for i in range(n_points):
        points[0][i] = [np.random.uniform(xBounds[0], xBounds[1]), np.random.uniform(yBounds[0], yBounds[1])]
    mixf = np.ones(t_steps)*partitionLoc/xmax             # Mixture fraction ~= particle density
    var = np.zeros(t_steps)

    # Simulate diffusion in a box
    for i in range(1, t_steps):
        if not partition[i]:
            xBounds = [0, xmax]
        xsteps = np.random.uniform(-1, 1, n_points) * maxStep
        ysteps = np.random.uniform(-1, 1, n_points) * maxStep
        points[i, :, 0] = points[i-1, :, 0] + xsteps
        points[i, :, 1] = points[i-1, :, 1] + ysteps
        points[i, :, 0] = np.clip(points[i, :, 0], xBounds[0], xBounds[1])
        points[i, :, 1] = np.clip(points[i, :, 1], yBounds[0], yBounds[1])
        var[i] = np.var(points[i, :, 0])

    # Mixture fraction variance is at its minimum (0) when the particles' variance is at its maximum
    # Mixture fraction variance is at its maximum (mixf*(1-mixf)) when the particles' variance is at its minimum
    var = (var-np.min(var))/(np.max(var) - np.min(var))  # Scale the variance to [0,1]
    maxVar = mixf*(1-mixf)                               # Maximum variance for each mixture fraction
    var = (1-var)*maxVar                                 # Scale the variance to the maximum variance

    # Animate the diffusion in time using matplotlib.animation
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, figsize=(8, 5))

    def update_plot(i):
        ax1.clear()
        ax2.clear()
        ax3.clear()
        
        ax1.scatter(points[i, :, 0], points[i, :, 1], marker = '.')
        if partition[i]:
            ax1.axvline(x=partitionLoc, color='r')
        ax1.set_title("Diffusion")
        ax1.set_xticks(xBounds)
        ax1.set_xticklabels([0, 1])
        ax1.set_xlim(xBounds)
        ax1.set_xlabel("x")
        ax1.set_ylim(yBounds)
        ax1.set_yticks([])
        ax1.set_ylabel("y", rotation=0, labelpad=15)
        ax1.set_aspect('equal')
        # Plot a histogram of the particle count
        ax2.hist(points[i, :, 0], bins=histBins, range=(0, xmax))
        ax2.set_xticks(xBounds)
        ax2.set_xticklabels([0, 1])
        ax2.set_xlim(xBounds)
        ax2.set_xlabel("x")
        ax2.set_ylabel("Frequency")
        ax2.set_yticks([])
        # Plot the beta PDF
        xs = np.linspace(0,1,10000)
        Ps = [LI.bPdf(x, mixf[i], var[i]) for x in xs]
        ax3.plot(xs, Ps)
        ax3.set_xlabel(r"Mixture Fraction ($\xi$)")
        ax3.set_xlim([-0.01, 1.01])
        ax3.set_ylabel(r"$P(\xi)$", rotation=0, labelpad=15)
        ax3.set_yticks([])
        fig.tight_layout()
        
    #update_plot(0)
    #plt.show()

    interval = 1000/fps  # ms per frame
    ani = animation.FuncAnimation(fig, update_plot, frames=t_steps, interval=interval, blit = False)
    ani.save("diffusion.gif", writer='pillow')

#### Simulation Result

In [29]:
HTML('<img src="diffusion.gif">')

## Binomial Distribution
* (useful for understanding beta distribution)
* Describes the probabilities of success or failure in an experiment with 2 possible values that is repeated multiple times. Examples:
    *  Simple: Coin toss. "How likely is it to toss heads 5/9 times?
    *  Complex: Sandwiches. "If there's a 70% likelihood that a given customer in the shop buys a chicken sandwich, what's the probability that 2/3 next customers choose chicken?"
*  Probability of a scenario where the desired outcome occurrs = $P(X) = p^k(1-p)^{(n-k)}$ where
    * **p** is the probability of the desired outcome (70%)
    * __k__ is the number of those outcomes we want (2)
    * **n** is the total number of trials (3)
* Possible # of scenarios with desired outcome = ${n \choose k} = \frac{n!}{k!(n-k)!} $ where
    * **k** is the number of outcomes we're selecting (2) from
    * **n** the total number of outcomes (3)
### Generalized Binomial Probability Formula:
   $$P(\mathrm{k~out~of~n}) = \frac{n!}{k!(n-k)!}p^k(1-p)^{(n-k)}$$

 * Variance: $\sigma^2 = np(1-p)$ where
     * n = Number of trials
     * p = Probability of success
 * Std. Deviation := $\sqrt{\sigma^2} = \sigma$

## Gamma Function
* Function "stitching together" all integer values of the factorial function:
$$ \Gamma(n) = (n-1)! = \int_0^\infty t^{z-1}e^{-t}dt $$
* The derivation of the formula is nasty, but the plot looks like this:
<div>
    <p style="text-align:center;"><img src = https://upload.wikimedia.org/wikipedia/commons/5/52/Gamma_plot.svg alt = "Gamma Function" width = "400" ></p>
</div>



## Beta Function
The Beta Function is defined as
$$ B(\alpha, \beta) = \int_0^1t^{\alpha-1}(1-t)^{\beta-1}dt = \frac{\Gamma(\alpha)\Gamma(\beta)}{\Gamma(\alpha + \beta)}$$
where $z_1, z_2 > 0$. It is useful for constructing the Beta Distribution:
## Beta Distribution
$$f(x; \alpha, \beta) = \frac{1}{B(\alpha, \beta)}~x^{\alpha-1}(1-x)^{\beta-1} = \frac{\Gamma(\alpha + \beta)}{\Gamma(\alpha)\Gamma(\beta)}~x^{\alpha-1}(1-x)^{\beta-1}$$
* Reading each piece:
    * $x^{\alpha-1}(1-x)^{\beta-1}$ describes the likelihood of x occurring
    * $\frac{\Gamma(\alpha + \beta)}{\Gamma(\alpha)\Gamma(\beta)} = \frac{1}{B(\alpha, \beta)}$ is a normalizing factor and ensures the total probability is 1
* Key difference between beta and binomial distributions: 
    * Binomial is a function of desired outcomes (fixed probability) and the number of possible outcomes
    * Beta is a function of the probability of two outcomes. 
* Conceptualizing: 
    * $\alpha - 1$ can be thought of as the # of observed successes
    * $\beta -1$ can be thought of as the # of observed failures
    * As $\alpha, \beta$ increase, the PDF narrows because of increased certainty about the system
![Animated Beta Distribution PDF](https://upload.wikimedia.org/wikipedia/commons/7/78/PDF_of_the_Beta_distribution.gif)

<div>
    <p style="text-align:center;"><img src = https://upload.wikimedia.org/wikipedia/commons/f/f3/Beta_distribution_pdf.svg alt = "Beta Distribution PDF - Set Values" width = "400" ></p>
</div>

[Beta Distribution Calculator](https://homepage.divms.uiowa.edu/~mbognar/applets/beta.html)



## *Sources:*
* https://towardsdatascience.com/beta-distribution-intuition-examples-and-derivation-cf00f4db57af
* https://en.wikipedia.org/wiki/Beta_distribution
* https://en.wikipedia.org/wiki/Beta_function
* https://en.wikipedia.org/wiki/Gamma_function